In [0]:
%tensorflow_version 1.x

In [0]:
#구글 드라이버를 마운트
from google.colab import drive

drive.mount('gdrive')

In [0]:
!ls -al "/content/gdrive/My Drive/multicampus_ai/semi_project/workspace/data/seq2seq/"

#데이터 읽어오기

* fra.txt 데이터는 영어 문장과 오른쪽의 프랑스어 문장 사이에 탭으로 구분되는 구조가 1줄
* 16만개의 병렬 문장 샘플을 포함
* src는 source의 줄임말로 입력 문장(영어)
* tar는 target의 줄임말로 번역하고자 하는 문장(불어)

In [0]:
import pandas as pd
lines= pd.read_csv('/content/gdrive/My Drive/multicampus_ai/semi_project/workspace/data/seq2seq/fra.txt', names=['src', 'tar',"desc"], sep='\t')
len(lines)

In [0]:
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

In [0]:
#desc 컬럼 삭제
lines.pop("desc")
lines.sample(10)

lines.tar :line의 tar 컬럼 의 각 줄의 앞에 tab("\t) 마지막에 엔터 ("\n) 을 추가해서 문장의 처음과 끝을 표시

In [0]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

#텍스트 데이터 전처리

In [0]:
# 글자 집합 구축
#set()은 리스트와 같이 복수의 데이터를 저장 할 수 있는객체
# 리스트와 다른 점은 이미 존재하는 데이터를 중복해서 추가시 나중에 추가되는 데이터는 추가하지 않고 무시
#예를 들어서 set객체에 1,2 가 저장되 있는데 1을 또다시 중복해서 추가하려고 하면 나중에 저장하려는 1은 추가되지 않고 무시
src_vocab=set()
#lines의 src컬럼을 1줄씩 읽어서 line에 대입
for line in lines.src:
    #line에 단어 1개씩 읽어서 char에 대입
    for char in line: 
        #src_vocab 에 char 추가 (중복 제거)
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [0]:
print("src_vocab:",src_vocab)
print("tar_vocab:",tar_vocab)

In [0]:
#입력글자의 수를 src_vocab_size에 대입
#입력 글자 수는 len(src_vocab) 전체 글자수 
# src_vocab에 서 학습하지 않은 글자는 모두 같은 글자로 취급해서 +1
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

In [0]:
# src_vocab 을 정렬
src_vocab = sorted(list(src_vocab))
#tar_vocab 을 정렬
tar_vocab = sorted(list(tar_vocab))
print("src_vocab:",src_vocab)
print("tar_vocab:",tar_vocab)

In [0]:
#for i, word in enumerate(src_vocab): src_vocab에 저장된 글자에 인덱스를 붙여서 리턴
#인덱스는 i 글자는 word에 저장

dict([(word, i+1) ..]
#단어를 key로  인덱스를 value로 사전 생성
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

In [0]:
encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
      #src_to_index 사전을 이용해서  글자를 해당되는 정수로 변환
      temp_X.append(src_to_index[w]) 
    encoder_input.append(temp_X)
    
print(encoder_input[:5])

In [0]:
#tar 문장의 글자들을 숫자로 변환
decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
      temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

In [0]:
#tar 문장의 두번째 글자 부터 숫자로 변환
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(tar_to_index[w])
      t=t+1
    decoder_target.append(temp_X)

print(decoder_target[:5])

In [0]:
#line in lines.src: lines.src 의 각 줄을 line에 저장 len(line) 
#len(line)의 글자수의 최대 값을 max_src_len에 대입
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

#incoder_input: 입력 글자가 숫자로 변환되서 저장된 배열 예)[30, 64, 10]
#max_src_len: incoder_input의 최대 글자수
#pad_sequences(encoder_input, maxlen=max_src_len, padding='post'): encoder_input 뒤에 (padding='post') 0을 붙여서 배열 크기를 max_src_len 으로 만들어줌
#예)[30, 64, 10] => [30,60,10,0,0,.....0]
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [0]:
from tensorflow.keras.utils import to_categorical
#to_categorical(encoder_input) :encoder_input을 one hot 인코딩함
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

#모델 생성

인코더 모델 생성

In [0]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

디코더 모델 생성

In [0]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

모델 학습 (epochs=3)

In [0]:
#model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=3, validation_split=0.2)

#모델 사용하기

In [0]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [0]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [0]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [0]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [0]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력